In [4]:

# %%
from tqdm import trange
import heapq
import random
import numpy as np
import copy
import time
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors  # Para colores
import os

from scipy import stats

# %%
# --- 1. Definición de Parámetros y Datos (Común) ---
setup_times = {
    1: 30,  # Tipo 1: Colecistectomía laparoscópica
    2: 30,  # Tipo 2: Colecistectomía
    3: 30   # Tipo 3: Drenaje T
}

cleanup_times = {
    1: 20,
    2: 20,
    3: 20
}

MAX_WAIT_TIMES = {
    1: 50,  # Anestesia
    2: 50,  # Cirugía
    3: 50   # Recuperación
}

job_types = {
    1: 1, 2: 1, 3: 1, 4: 3, 5: 3, 6: 3, 7: 2, 8: 2, 9: 1,
    10: 1, 11: 2, 12: 2, 13: 3, 14: 1, 15: 3
}
base_day_surgeries_data = {
    1: {1: 30, 2: 60, 3: 40},
    2: {1: 40, 2: 60, 3: 40},
    3: {1: 35, 2: 80, 3: 40},
    4: {1: 65, 2: 190, 3: 60},
    5: {1: 70, 2: 190, 3: 60},
    6: {1: 75, 2: 190, 3: 60},
    7: {1: 80, 2: 150, 3: 50},
    8: {1: 70, 2: 110, 3: 50},
    9: {1: 35, 2: 80, 3: 40},
    10: {1: 30, 2: 80, 3: 40},
    11: {1: 60, 2: 110, 3: 50},
    12: {1: 80, 2: 110, 3: 50},
    13: {1: 65, 2: 210, 3: 60},
    14: {1: 40, 2: 70, 3: 40},
    15: {1: 70, 2: 160, 3: 60}
}

def generate_day_surgeries_data(job_ids, base_data, std_factor):
    """Genera datos basados en los valores base de la Tabla 5"""
    data = {}
    for j in job_ids:
        if j in base_data:
            # Usar valores base directamente
            data[j] = copy.deepcopy(base_data[j])

            # Añadir variabilidad si es necesario (std_factor > 0)
            if std_factor > 0:
                for op in [1, 2, 3]:
                    base_val = base_data[j][op]
                    std_val = std_factor * base_val
                    value = np.random.normal(base_val, std_val)
                    data[j][op] = max(1, round(value, 2))
        else:
            # Valor por defecto para trabajos no definidos
            data[j] = {1: 30, 2: 60, 3: 40}
    return data

emergency_jobs = []
job_ids = list(job_types.keys())
num_jobs = len(job_ids)
num_aprs = 4
num_ors = 4
num_arrs = 4

aprs = [f'APR_{i+1}' for i in range(num_aprs)]
ors = [f'OR_{i+1}' for i in range(num_ors)]
arrs = [f'ARR_{i+1}' for i in range(num_arrs)]
all_rooms = aprs + ors + arrs  # Mantener este orden para el Gantt

personnel_map = {
    1: {1: 'A6', 2: 'S1', 3: 'D1'},
    2: {1: 'A6', 2: 'S2', 3: 'D2'},
    3: {1: 'A2', 2: 'S3', 3: 'D3'},
    4: {1: 'A2', 2: 'S3', 3: 'D3'},
    5: {1: 'A2', 2: 'S2', 3: 'D2'},
    6: {1: 'A5', 2: 'S1', 3: 'D1'},
    7: {1: 'A5', 2: 'S2', 3: 'D2'},
    8: {1: 'A4', 2: 'S3', 3: 'D3'},
    9: {1: 'A1', 2: 'S3', 3: 'D3'},
    10: {1: 'A1', 2: 'S1', 3: 'D1'},
    11: {1: 'A4', 2: 'S1', 3: 'D1'},
    12: {1: 'A4', 2: 'S2', 3: 'D2'},
    13: {1: 'A1', 2: 'S3', 3: 'D3'},
    14: {1: 'A3', 2: 'S2', 3: 'D2'},
    15: {1: 'A3', 2: 'S2', 3: 'D2'}
}
all_personnel = ([f'A{i}' for i in range(1, 7)] +
                 [f'S{i}' for i in range(1, 4)] +
                 [f'D{i}' for i in range(1, 4)])
alpha = 1e-6  # Factor de ponderación para suma de tiempos de inicio
alpha_test = 0.05 # Nivel de significancia para pruebas estadísticas

# %%
# --- 2. Parámetros Específicos ---

# GA Parameters
POPULATION_SIZE_GA = 30
MAX_GENERATIONS = 1000
CROSSOVER_PROBABILITY = 0.8
MUTATION_PROBABILITY = 0.3
ELITISM_COUNT = 2

# dPSO Parameters (Defaults)
SWARM_SIZE_DPSO = 30
MAX_ITERATIONS_DPSO = 1000
W_DPSO = 0.7  # Inercia
C1_DPSO = 1.5 # Coeficiente cognitivo
C2_DPSO = 1.5 # Coeficiente social
VEL_HIGH_DPSO = 4.0 # Límite superior de velocidad
VEL_LOW_DPSO = -VEL_HIGH_DPSO # Límite inferior de velocidad

# Dimensiones para dPSO (si se usaran directamente)
dim_sequence = num_jobs
dim_rooms = num_jobs * 3
dim_total = dim_sequence + dim_rooms

# Mapeos útiles
job_id_to_index = {job_id: i for i, job_id in enumerate(job_ids)}
index_to_job_id = {i: job_id for i, job_id in enumerate(job_ids)}

# %%
# --- 3. Funciones de Cálculo de Makespan (CORREGIDAS y REFORMATADAS) ---

def calculate_makespan_ga(individual, surgeries_data, personnel_map, return_details=False):
    """
    Calcula el fitness (objetivo combinado) para un individuo del GA.
    Simula la programación aplicando la restricción de bloqueo "no buffer".
    Opcionalmente, devuelve detalles de la programación.
    CORREGIDO: Re-calcula el tiempo de inicio justo antes de programar.
    """
    # --- Extracción de Datos ---
    job_sequence_base = individual['job_sequence_base']
    room_assignment = individual['room_assignment']
    current_job_ids = list(individual.get('room_assignment', {}).keys())

    if not current_job_ids:
        return (0, 0, []) if return_details else 0

    total_ops = 3 * len(current_job_ids)

    # --- Inicialización de Estado ---
    room_release_time = {room: 0 for room in all_rooms}
    personnel_release_time = {pers: 0 for pers in all_personnel}
    # Tiempos de fin de *procesamiento* (sin cleanup) por job y operación
    job_op_processing_end = {job: {0: 0} for job in current_job_ids}
    # Tiempos de fin de *máquina* (con cleanup) por job y operación
    job_op_machine_end = {job: {0: 0} for job in current_job_ids}
    # Tiempos de inicio por job y operación
    job_op_start = {job: {0: 0} for job in current_job_ids}
    # Recursos usados por job y operación (sala, personal)
    job_op_used_res = {job: {0: (None, None)} for job in current_job_ids}
    # Siguiente operación a programar para cada job
    next_op_num = {job: 1 for job in current_job_ids}
    ops_done = 0
    # Prioridad basada en la secuencia base del individuo
    job_priority = {job: i for i, job in enumerate(job_sequence_base)}
    schedule_details = []

    # --- Bucle Principal de Simulación ---
    # Cola de prioridad: (start_time_estimado, prioridad_job, job_id, op_num)
    possible_ops = []
    # scheduled_this_iteration = {} # Tracker (opcional, quitado por simplicidad ahora)

    # Inicializar cola con las primeras operaciones (op=1) posibles
    for job in current_job_ids:
        op = next_op_num.get(job)
        if op == 1:
            prev_end = job_op_processing_end[job][0]  # = 0
            if job not in room_assignment or op not in room_assignment[job]:
                return (float('inf'), float('inf'), None) if return_details else float('inf')
            assigned_room = room_assignment[job][op]
            if job not in personnel_map or op not in personnel_map[job]:
                return (float('inf'), float('inf'), None) if return_details else float('inf')
            assigned_personnel = personnel_map[job][op]

            room_avail = room_release_time.get(assigned_room, 0)
            pers_avail = personnel_release_time.get(assigned_personnel, 0)
            start_time = max(prev_end, room_avail, pers_avail)
            heapq.heappush(possible_ops, (start_time, job_priority[job], job, op))

    while ops_done < total_ops:
        # Si la cola está vacía pero no hemos terminado -> Error/Deadlock
        if not possible_ops:
            valid_remaining = any(next_op_num.get(j, 4) <= 3 for j in current_job_ids)
            if valid_remaining or ops_done < total_ops:
                return (float('inf'), float('inf'), None) if return_details else float('inf')
            else:
                break

        # 2. Seleccionar la Mejor Operación Candidata (la sacamos de la cola)
        _, _, best_job, best_op = heapq.heappop(possible_ops)

        # --- RE-VALIDACIÓN y RE-CÁLCULO ---
        # Obtener recursos asignados y disponibilidad ACTUAL
        if best_job not in room_assignment or best_op not in room_assignment[best_job]:
            return (float('inf'), float('inf'), None) if return_details else float('inf')
        assigned_room = room_assignment[best_job][best_op]
        if best_job not in personnel_map or best_op not in personnel_map[best_job]:
            return (float('inf'), float('inf'), None) if return_details else float('inf')
        assigned_personnel = personnel_map[best_job][best_op]

        prev_op = best_op - 1

        # Asegurarse que el estado del predecesor existe
        if best_job not in job_op_processing_end or prev_op not in job_op_processing_end[best_job]:
            return (float('inf'), float('inf'), None) if return_details else float('inf')
        prev_end = job_op_processing_end[best_job][prev_op] # Lo mantenemos por si acaso, pero no lo usaremos en max


        current_room_release = room_release_time.get(assigned_room, 0)
        current_pers_release = personnel_release_time.get(assigned_personnel, 0)

        # Calcular el tiempo de inicio REAL posible AHORA
        actual_start_time = max(prev_end, current_room_release, current_pers_release)

        # 3. Programar la Operación Seleccionada con actual_start_time
        if best_job not in surgeries_data or best_op not in surgeries_data[best_job]:
            return (float('inf'), float('inf'), None) if return_details else float('inf')
        processing_time = surgeries_data[best_job][best_op]

        # Calcular tiempos de setup y cleanup
        # Obtener tiempos específicos por tipo de cirugía
        job_type = job_types[best_job]
        setup_time_val = setup_times[job_type]  # Setup por tipo
        cleanup_time_val = cleanup_times[job_type]  # Cleanup por tipo

        # Calcular tiempos usando actual_start_time
        proc_end = actual_start_time + setup_time_val + processing_time
        # Tiempo hasta que la sala/personal quedan realmente libres
        finish = proc_end + cleanup_time_val

        total_time_in_resource = finish - actual_start_time
        processing_time_only = proc_end - (actual_start_time + setup_time_val)
        waiting_time = total_time_in_resource - processing_time_only

        if waiting_time > MAX_WAIT_TIMES[best_op]:
            return (float('inf'), float('inf'), None) if return_details else float('inf')

        # Guardar tiempos y recursos usados
        job_op_start[best_job][best_op] = actual_start_time
        job_op_processing_end[best_job][best_op] = proc_end
        job_op_machine_end[best_job][best_op] = finish
        job_op_used_res[best_job][best_op] = (assigned_room, assigned_personnel)

        # Guardar detalles para Gantt si se solicitan
        if return_details:
            schedule_details.append({
                'Job': best_job,
                'Operation': best_op,
                'Resource': assigned_room,
                'Personnel': assigned_personnel,
                'Start': actual_start_time,
                'ProcessingEnd': proc_end,
                'Finish': finish
            })

        # 4. Actualizar Tiempos de Liberación de Recursos
        # a) Recursos propios (best_op): Se liberan al tiempo 'finish'

        if assigned_room:
            room_release_time[assigned_room] = finish
        if assigned_personnel:
            personnel_release_time[assigned_personnel] = finish

        # b) Recursos del predecesor (best_op - 1) - Lógica de bloqueo:
        #    Se liberan realmente cuando la operación actual (best_op) empieza.

        if best_op > 1:
            prev_op_num = best_op - 1
            prev_res = job_op_used_res[best_job].get(prev_op_num)
            if prev_res and prev_res != (None, None):
                prev_room, prev_personnel = prev_res
                # Tiempo real de liberación = inicio de la operación actual
                actual_release_prev = actual_start_time
                # Actualizar liberación de recursos PREVIOS,
                # asegurando no retroceder el tiempo si ya fue actualizado
                if prev_room:
                    room_release_time[prev_room] = max(
                        room_release_time.get(prev_room, 0), actual_release_prev
                    )
                if prev_personnel:
                    personnel_release_time[prev_personnel] = max(
                        personnel_release_time.get(prev_personnel, 0), actual_release_prev
                    )

        # 5. Avanzar Estado y Añadir Siguiente Operación a la Cola

        next_op_num[best_job] = next_op_num.get(best_job, 0) + 1
        ops_done += 1

        # Añadir la SIGUIENTE operación de ESTE job a la cola si existe
        next_op = next_op_num.get(best_job)
        if next_op is not None and next_op <= 3:
            # Validar datos necesarios para la siguiente operación
            if best_job not in room_assignment or next_op not in room_assignment[best_job]:
                return (float('inf'), float('inf'), None) if return_details else float('inf')
            next_assigned_room = room_assignment[best_job][next_op]
            if best_job not in personnel_map or next_op not in personnel_map[best_job]:
                return (float('inf'), float('inf'), None) if return_details else float('inf')
            next_assigned_personnel = personnel_map[best_job][next_op]

            # Calcular EST (Earliest Start Time) para la siguiente operación
            # Necesita el fin de procesamiento de la operación actual (best_op)
            next_prev_end = job_op_processing_end[best_job][best_op]
            next_room_avail = room_release_time.get(next_assigned_room, 0)
            next_pers_avail = personnel_release_time.get(next_assigned_personnel, 0)

            # El EST considera fin del predecesor y disponibilidad ACTUAL de recursos
            next_start_time = max(next_prev_end, next_room_avail, next_pers_avail)

            # Añadir a la cola de prioridad
            heapq.heappush(
                possible_ops,
                (next_start_time, job_priority[best_job], best_job, next_op)
            )

    # --- Fin del Bucle ---

    # Calcular Makespan Final (tiempo de fin de la última operación de todos los jobs)
    final_makespan = 0
    if current_job_ids:
        try:
            final_makespan = max(job_op_machine_end.get(j, {}).get(3, 0) for j in current_job_ids)
        except: # Captura cualquier error potencial en el max() si hay datos raros
            final_makespan = float('inf')

    # Chequeo de validez final
    if final_makespan == 0 and total_ops > 0:
        return (float('inf'), float('inf'), None) if return_details else float('inf')
    if final_makespan == float('inf'):
        return (float('inf'), float('inf'), None) if return_details else float('inf')

    # Calcular Suma de Tiempos de Inicio (para objetivo secundario)
    total_start = sum(job_op_start.get(j, {}).get(op, 0)
                    for j in current_job_ids
                    for op in [1, 2, 3]
                    if job_op_start.get(j, {}).get(op, -1) >= 0) # Solo ops que iniciaron

    # Objetivo Combinado
    combined_obj = final_makespan + alpha * total_start

    # Devolver resultados
    if return_details:
        details_to_return = schedule_details if final_makespan != float('inf') else None
        return combined_obj, final_makespan, details_to_return
    else:
        return combined_obj

def calculate_makespan_dpso(particle, surgeries_data, personnel_map, return_details=False):
    """
    Calcula el fitness para una partícula dPSO usando su posición discreta.
    CORREGIDO: Re-calcula el tiempo de inicio justo antes de programar.
    (Es idéntica a la versión GA, pero toma 'particle' como input)
    """
    individual = particle.get_discrete_position_representation()

    # --- Extracción de Datos ---
    job_sequence_base = individual['job_sequence_base']
    room_assignment = individual['room_assignment']
    current_job_ids = particle.job_ids # Usar los job_ids de la partícula

    if not current_job_ids:
        return (0, 0, []) if return_details else 0

    total_ops = 3 * len(current_job_ids)

    # --- Inicialización de Estado ---
    room_release_time = {room: 0 for room in all_rooms}
    personnel_release_time = {pers: 0 for pers in all_personnel}
    job_op_processing_end = {job: {0: 0} for job in current_job_ids}
    job_op_machine_end = {job: {0: 0} for job in current_job_ids}
    job_op_start = {job: {0: 0} for job in current_job_ids}
    job_op_used_res = {job: {0: (None, None)} for job in current_job_ids}
    next_op_num = {job: 1 for job in current_job_ids}
    ops_done = 0
    job_priority = {job: i for i, job in enumerate(job_sequence_base)}
    schedule_details = []
    tolerance = 1e-9

    # --- Bucle Principal de Simulación ---
    possible_ops = []
    # scheduled_this_iteration = {}

    # Inicializar cola con las primeras operaciones posibles
    for job in current_job_ids:
        op = next_op_num.get(job)
        if op == 1:
            # Validar asignaciones antes de usarlas
            if job not in room_assignment or op not in room_assignment[job]:
                # print(f"Error dPSO Init: Falta asignación sala J:{job} O:{op}")
                return (float('inf'), float('inf'), None) if return_details else float('inf')
            assigned_room = room_assignment[job][op]
            if job not in personnel_map or op not in personnel_map[job]:
                # print(f"Error dPSO Init: Falta asignación personal J:{job} O:{op}")
                return (float('inf'), float('inf'), None) if return_details else float('inf')
            assigned_personnel = personnel_map[job][op]

            prev_end = job_op_processing_end[job][0]  # = 0
            room_avail = room_release_time.get(assigned_room, 0)
            pers_avail = personnel_release_time.get(assigned_personnel, 0)
            start_time = max(prev_end, room_avail, pers_avail)
            heapq.heappush(possible_ops, (start_time, job_priority[job], job, op))
            # scheduled_this_iteration[(job, op)] = start_time

    while ops_done < total_ops:
        if not possible_ops:
            valid_remaining = any(next_op_num.get(j, 4) <= 3 for j in current_job_ids)
            if valid_remaining or ops_done < total_ops:
                # print(f"Error/Deadlock dPSO: No ops posibles. Quedan {total_ops - ops_done}. Ops done: {ops_done}")
                return (float('inf'), float('inf'), None) if return_details else float('inf')
            else:
                break

        _, _, best_job, best_op = heapq.heappop(possible_ops)

        # --- RE-VALIDACIÓN y RE-CÁLCULO ---
        if best_job not in room_assignment or best_op not in room_assignment[best_job]:
            return (float('inf'), float('inf'), None) if return_details else float('inf')
        assigned_room = room_assignment[best_job][best_op]
        if best_job not in personnel_map or best_op not in personnel_map[best_job]:
            return (float('inf'), float('inf'), None) if return_details else float('inf')
        assigned_personnel = personnel_map[best_job][best_op]

        prev_op = best_op - 1
        if best_job not in job_op_processing_end or prev_op not in job_op_processing_end[best_job]:
            return (float('inf'), float('inf'), None) if return_details else float('inf')
        prev_end = job_op_processing_end[best_job][prev_op]

        current_room_release = room_release_time.get(assigned_room, 0)
        current_pers_release = personnel_release_time.get(assigned_personnel, 0)

        actual_start_time = max(prev_end, current_room_release, current_pers_release)

        # 3. Programar la Operación Seleccionada
        if best_job not in surgeries_data or best_op not in surgeries_data[best_job]:
            return (float('inf'), float('inf'), None) if return_details else float('inf')
        processing_time = surgeries_data[best_job][best_op]

        job_type = job_types[best_job]
        setup_time_val = setup_times[job_type]
        cleanup_time_val = cleanup_times[job_type]

        proc_end = actual_start_time + setup_time_val + processing_time
        finish = proc_end + cleanup_time_val

        job_op_start[best_job][best_op] = actual_start_time
        job_op_processing_end[best_job][best_op] = proc_end
        job_op_machine_end[best_job][best_op] = finish
        job_op_used_res[best_job][best_op] = (assigned_room, assigned_personnel)

        if return_details:
            schedule_details.append({
                'Job': best_job,
                'Operation': best_op,
                'Resource': assigned_room,
                'Personnel': assigned_personnel,
                'Start': actual_start_time,
                'ProcessingEnd': proc_end,
                'Finish': finish
            })

        # 4. Actualizar Tiempos de Liberación de Recursos
        if assigned_room:
            room_release_time[assigned_room] = finish
        if assigned_personnel:
            personnel_release_time[assigned_personnel] = finish

        if best_op > 1:
            prev_op_num = best_op - 1
            prev_res = job_op_used_res[best_job].get(prev_op_num)
            if prev_res and prev_res != (None, None):
                prev_room, prev_personnel = prev_res
                actual_release_prev = actual_start_time
                if prev_room:
                    room_release_time[prev_room] = max(
                        room_release_time.get(prev_room, 0), actual_release_prev
                    )
                if prev_personnel:
                    personnel_release_time[prev_personnel] = max(
                        personnel_release_time.get(prev_personnel, 0), actual_release_prev
                    )

        # 5. Avanzar Estado y Añadir Siguiente Operación a la Cola
        next_op_num[best_job] = next_op_num.get(best_job, 0) + 1
        ops_done += 1

        next_op = next_op_num.get(best_job)
        if next_op is not None and next_op <= 3:
            if best_job not in room_assignment or next_op not in room_assignment[best_job]:
                return (float('inf'), float('inf'), None) if return_details else float('inf')

            next_assigned_room = room_assignment[best_job][next_op]

            if best_job not in personnel_map or next_op not in personnel_map[best_job]:
                return (float('inf'), float('inf'), None) if return_details else float('inf')

            next_assigned_personnel = personnel_map[best_job][next_op]
            next_prev_end = job_op_processing_end[best_job][best_op]
            next_room_avail = room_release_time.get(next_assigned_room, 0)
            next_pers_avail = personnel_release_time.get(next_assigned_personnel, 0)
            next_start_time = max(next_prev_end, next_room_avail, next_pers_avail)

            heapq.heappush(
                possible_ops,
                (next_start_time, job_priority[best_job], best_job, next_op)
            )

    # --- Fin del Bucle ---
    final_makespan = 0
    if current_job_ids:
        try:
            final_makespan = max(job_op_machine_end.get(j, {}).get(3, 0) for j in current_job_ids)
        except:
            final_makespan = float('inf')

    if final_makespan == 0 and total_ops > 0:
        return (float('inf'), float('inf'), None) if return_details else float('inf')
    if final_makespan == float('inf'):
        return (float('inf'), float('inf'), None) if return_details else float('inf')

    total_start = sum(job_op_start.get(j, {}).get(op, 0)
                    for j in current_job_ids
                    for op in [1, 2, 3]
                    if job_op_start.get(j, {}).get(op, -1) >= 0)

    combined_obj = final_makespan + alpha * total_start

    if return_details:
        details_to_return = schedule_details if final_makespan != float('inf') else None
        return combined_obj, final_makespan, details_to_return
    else:
        return combined_obj

# %%
# --- 4. Funciones Específicas del GA ---

def create_individual(job_ids):
    """Crea un individuo aleatorio para el GA según especificación del paper."""
    individual = {}
    # 1. Secuencia base aleatoria de trabajos (como antes)
    base_sequence = random.sample(job_ids, len(job_ids))

    # 2. CORRECCIÓN CRÍTICA (Sección 4.1 del paper):
    #    La capa 1 debe tener TRES COPIA IDÉNTICAS de la secuencia base
    individual['job_sequence_base'] = base_sequence * 3

    # 3. Asignación de salas (igual que antes)
    individual['room_assignment'] = {
        job: {
            1: random.choice(aprs),
            2: random.choice(ors),
            3: random.choice(arrs)
        } for job in job_ids
    }
    return individual

def selection(population, fitnesses):
    """Selección por ruleta invertida basada en fitness (minimización)."""
    # Filtrar individuos con fitness infinito (inválidos)
    valid_indices = [i for i, f in enumerate(fitnesses) if f != float('inf')]

    # Si no hay válidos, devolver una población nueva aleatoria
    if not valid_indices:
        return [create_individual(job_ids) for _ in range(len(population))]

    valid_pop = [population[i] for i in valid_indices]
    valid_fit = [fitnesses[i] for i in valid_indices]

    # Invertir fitness para selección (mejor fitness -> mayor probabilidad)
    # Añadir 1 para evitar división por cero si todos tienen el mismo max fitness
    max_fit = max(valid_fit) + 1
    inverted_fitness = [(max_fit - f) for f in valid_fit]
    total_inverted_fitness = sum(inverted_fitness)

    # Si el total es 0 (todos los válidos tienen el mismo fitness máximo), elegir aleatoriamente
    if total_inverted_fitness == 0:
        return random.choices(valid_pop, k=len(population))

    # Calcular probabilidades de selección
    probabilities = [f / total_inverted_fitness for f in inverted_fitness]
    probabilities_sum = sum(probabilities)

    # Normalizar probabilidades (por si hay errores de precisión) y elegir
    if probabilities_sum <= 0:
        # Fallback: elegir aleatoriamente si las probabilidades son inválidas
        chosen_indices_valid = random.choices(range(len(valid_pop)), k=len(population))
    else:
        # Normalizar
        probabilities = np.array(probabilities) / probabilities_sum
        # Elegir índices con reemplazo según probabilidades
        chosen_indices_valid = np.random.choice(
            len(valid_pop),
            size=len(population),
            replace=True,
            p=probabilities
        )

    # Devolver la nueva población seleccionada
    return [valid_pop[i] for i in chosen_indices_valid]

def crossover(parent1, parent2):
    """Realiza crossover en secuencia (Order Crossover - OX1) y asignación de salas."""
    child1 = copy.deepcopy(parent1)
    child2 = copy.deepcopy(parent2)

    # Crossover de secuencia con probabilidad CROSSOVER_PROBABILITY
    if random.random() < CROSSOVER_PROBABILITY:
        seq1 = parent1['job_sequence_base']
        seq2 = parent2['job_sequence_base']
        n = len(seq1)

        if n >= 2:
            # Elegir dos puntos de corte
            p1, p2 = sorted(random.sample(range(n), 2))

            # Order Crossover (OX1) para Child 1
            sub1 = seq1[p1:p2+1]
            remaining1 = [item for item in seq2 if item not in sub1]
            child1['job_sequence_base'] = remaining1[-(n-(p2+1)):] + sub1 + remaining1[:-(n-(p2+1))]

            # Order Crossover (OX1) para Child 2
            sub2 = seq2[p1:p2+1]
            remaining2 = [item for item in seq1 if item not in sub2]
            child2['job_sequence_base'] = remaining2[-(n-(p2+1)):] + sub2 + remaining2[:-(n-(p2+1))]

    # Crossover de asignación de salas (punto de corte simple)
    # (Se hace independientemente de la secuencia para probar más combinaciones)
    if random.random() < CROSSOVER_PROBABILITY:
        jobs_list = list(parent1['room_assignment'].keys())
        if len(jobs_list) > 1:
            cut_point = random.randint(1, len(jobs_list) - 1)
            jobs_to_swap = jobs_list[cut_point:]
            for job in jobs_to_swap:
                if job in parent1['room_assignment'] and job in parent2['room_assignment']:
                    child1['room_assignment'][job] = parent2['room_assignment'][job]
                    child2['room_assignment'][job] = parent1['room_assignment'][job]
                # Si un padre no tiene el job (caso raro), mantener el original
                elif job in parent1['room_assignment']:
                    child2['room_assignment'][job] = parent1['room_assignment'][job]
                elif job in parent2['room_assignment']:
                    child1['room_assignment'][job] = parent2['room_assignment'][job]

    return child1, child2


def mutate(individual):
    """Realiza mutación en secuencia (swap) y asignación de salas."""
    ind = copy.deepcopy(individual)

    # Mutación de secuencia (swap)
    if random.random() < MUTATION_PROBABILITY:
        seq = ind['job_sequence_base']
        if len(seq) >= 2:
            i1, i2 = random.sample(range(len(seq)), 2)
            seq[i1], seq[i2] = seq[i2], seq[i1]

    # Mutación de asignación de salas (cambio aleatorio por operación)
    jobs_list = list(ind.get('room_assignment', {}).keys())
    # Probabilidad de mutar *cada asignación de sala*
    # Ajustar si MUTATION_PROBABILITY es muy alta/baja
    mutation_prob_per_room = MUTATION_PROBABILITY / 3 # Dividido entre 3 operaciones

    for job in jobs_list:
        if job not in ind['room_assignment']: continue # Skip si falta el job
        for op in [1, 2, 3]:
            if op not in ind['room_assignment'][job]: continue # Skip si falta la op
            if random.random() < mutation_prob_per_room:
                if op == 1:
                    ind['room_assignment'][job][op] = random.choice(aprs)
                elif op == 2:
                    ind['room_assignment'][job][op] = random.choice(ors)
                else:  # op == 3
                    ind['room_assignment'][job][op] = random.choice(arrs)
    return ind

def sigmoid(x):
    """Función sigmoide para mapear velocidad a probabilidad."""
    # Recortar x para evitar overflow en np.exp
    clipped_x = np.clip(x, -500, 500)
    return 1 / (1 + np.exp(-clipped_x))

class DiscreteParticle:
    """Representa una partícula en el espacio discreto de dPSO."""
    def __init__(self, job_ids, aprs, ors, arrs, vel_low, vel_high):
        self.job_ids = job_ids
        self.num_jobs = len(job_ids)
        self.aprs = aprs
        self.ors = ors
        self.arrs = arrs

        # Posición discreta
        self.position_sequence = random.sample(self.job_ids, self.num_jobs)
        self.position_rooms = {
            job: {
                1: random.choice(self.aprs),
                2: random.choice(self.ors),
                3: random.choice(self.arrs)
            } for job in self.job_ids
        }

        # Velocidad continua (una para secuencia, otra para salas)
        self.velocity_seq = np.random.uniform(vel_low, vel_high, self.num_jobs)
        # Velocidad para salas (vector plano: job1_op1, job1_op2, ..., jobN_op3)
        self.velocity_rooms_flat = np.random.uniform(vel_low, vel_high, self.num_jobs * 3)

        # Mejor posición personal encontrada
        self.pbest_sequence = self.position_sequence[:] # Copia
        self.pbest_rooms = copy.deepcopy(self.position_rooms)
        self.pbest_value = float('inf')

        # Valor actual (fitness)
        self.current_value = float('inf')

    def get_discrete_position_representation(self):
        """Devuelve la posición actual en el formato usado por makespan funcs."""
        return {
            'job_sequence_base': self.position_sequence,
            'room_assignment': self.position_rooms
        }


# %%
# --- 8. Funciones Principales de Ejecución ---

def run_genetic_algorithm(surgeries_data, personnel_map, job_ids, num_jobs, seed):
    """Ejecuta el algoritmo genético."""
    random.seed(seed)
    np.random.seed(seed)

    # Inicializar población
    population = [create_individual(job_ids) for _ in range(POPULATION_SIZE_GA)]

    best_objective_overall = float('inf')
    best_solution_overall = None
    best_fitness_history = []
    avg_fitness_history = []

    print(f"GA - Gen 0...", end="")
    for generation in range(MAX_GENERATIONS):
        if (generation + 1) % 200 == 0:
            print(f" {generation + 1}...", end="")

        # Calcular fitness de la población actual
        fitnesses = [
            calculate_makespan_ga(ind, surgeries_data, personnel_map)
            for ind in population
        ]

        # --- Seguimiento de historial ---
        valid_indices = [i for i, f in enumerate(fitnesses) if f != float('inf')]
        if not valid_indices:
            best_fitness_gen = float('inf')
            avg_fitness_gen = float('inf')
        else:
            valid_fitnesses = [fitnesses[i] for i in valid_indices]
            best_fitness_gen = min(valid_fitnesses)
            avg_fitness_gen = np.mean(valid_fitnesses)

        # Guardar mejor hasta ahora (evitando que suba si la generación empeora)
        best_fitness_history.append(
            min(best_fitness_gen, best_fitness_history[-1])
            if best_fitness_history and best_fitness_history[-1] != float('inf')
            else best_fitness_gen
        )
        # Guardar promedio (si no hay válido, usar el anterior o inf)
        avg_fitness_history.append(
            avg_fitness_gen if avg_fitness_gen != float('inf')
            else (avg_fitness_history[-1] if avg_fitness_history else float('inf'))
        )
        # --- Fin Seguimiento ---

        # Actualizar mejor solución global encontrada
        current_best_gen_val = min(fitnesses) # Incluye Inf si no hay válidos
        if current_best_gen_val < best_objective_overall:
            best_objective_overall = current_best_gen_val
            try:
                # Obtener el índice del mejor fitness
                best_index = fitnesses.index(best_objective_overall)
                best_solution_overall = copy.deepcopy(population[best_index])
            except ValueError:
                # Caso raro: si min(fitnesses) no está en fitnesses (debería estar)
                best_solution_overall = None

        # --- Crear Nueva Generación ---
        # 1. Elitismo: Conservar los N mejores individuos
        # Ordenar por fitness (ascendente), ignorando infinitos
        sorted_pop_indices = np.argsort(fitnesses)
        elite = [
            copy.deepcopy(population[i]) for i in sorted_pop_indices
            if fitnesses[i] != float('inf')
        ][:ELITISM_COUNT]

        # 2. Selección: Elegir padres para la próxima generación
        selected_population = selection(population, fitnesses)

        # 3. Crossover y Mutación
        next_population = elite # Empezar con la élite
        pop_indices_for_pairing = list(range(len(selected_population)))

        while len(next_population) < POPULATION_SIZE_GA:
            # Elegir dos padres distintos si es posible
            if len(pop_indices_for_pairing) >= 2:
                idx1, idx2 = random.sample(pop_indices_for_pairing, 2)
                parent1 = selected_population[idx1]
                parent2 = selected_population[idx2]
            elif len(pop_indices_for_pairing) == 1: # Reusar el único padre
                parent1 = selected_population[pop_indices_for_pairing[0]]
                parent2 = selected_population[pop_indices_for_pairing[0]] # O crear uno nuevo
            else: # Si no hay padres seleccionados (raro), crear nuevos
                parent1 = create_individual(job_ids)
                parent2 = create_individual(job_ids)

            # Realizar crossover
            child1, child2 = crossover(parent1, parent2)

            # Añadir hijos mutados a la nueva población
            next_population.append(mutate(child1))
            if len(next_population) < POPULATION_SIZE_GA:
                next_population.append(mutate(child2))

        # Reemplazar la población antigua por la nueva
        population = next_population[:POPULATION_SIZE_GA] # Asegurar tamaño exacto

    print(" ¡Hecho!") # Fin de las generaciones
    return best_objective_overall, best_solution_overall, best_fitness_history, avg_fitness_history

def run_dpso_algorithm(surgeries_data, personnel_map, job_ids, num_jobs, seed):
    """Ejecuta el algoritmo Discrete Particle Swarm Optimization (dPSO)."""
    random.seed(seed)
    np.random.seed(seed)

    # Inicializar enjambre (swarm)
    swarm = [
        DiscreteParticle(job_ids, aprs, ors, arrs, VEL_LOW_DPSO, VEL_HIGH_DPSO)
        for _ in range(SWARM_SIZE_DPSO)
    ]

    # Mejor global (gbest)
    gbest_sequence = None
    gbest_rooms = None
    gbest_value = float('inf')

    best_fitness_history = []
    avg_fitness_history = []

    print(f"dPSO - Iter 0...", end="")
    for iteration in range(MAX_ITERATIONS_DPSO):
        if (iteration + 1) % 200 == 0:
            print(f" {iteration + 1}...", end="")

        current_fitnesses = []
        # Evaluar cada partícula y actualizar pbest/gbest
        for i, particle in enumerate(swarm):
            fitness = calculate_makespan_dpso(particle, surgeries_data, personnel_map)
            particle.current_value = fitness
            current_fitnesses.append(fitness)

            # Actualizar pbest (mejor personal)
            if fitness < particle.pbest_value:
                particle.pbest_value = fitness
                particle.pbest_sequence = particle.position_sequence[:] # Copia
                particle.pbest_rooms = copy.deepcopy(particle.position_rooms)

            # Actualizar gbest (mejor global)
            if fitness < gbest_value:
                gbest_value = fitness
                gbest_sequence = particle.position_sequence[:] # Copia
                gbest_rooms = copy.deepcopy(particle.position_rooms)

        # --- Seguimiento de historial ---
        valid_fitnesses = [f for f in current_fitnesses if f != float('inf')]
        if not valid_fitnesses:
            best_fitness_iter = float('inf')
            avg_fitness_iter = float('inf')
        else:
            # Usar gbest si es válido, si no, el mínimo de la iteración
            best_fitness_iter = gbest_value if gbest_value != float('inf') else min(valid_fitnesses)
            avg_fitness_iter = np.mean(valid_fitnesses)

        best_fitness_history.append(
            min(best_fitness_iter, best_fitness_history[-1])
            if best_fitness_history and best_fitness_history[-1] != float('inf')
            else best_fitness_iter
        )
        avg_fitness_history.append(
            avg_fitness_iter if avg_fitness_iter != float('inf')
            else (avg_fitness_history[-1] if avg_fitness_history else float('inf'))
        )
        # --- Fin Seguimiento ---

        # Si no se ha encontrado ninguna solución válida aún, continuar
        if gbest_sequence is None:
            continue

        # Actualizar velocidad y posición de cada partícula
        for particle in swarm:
            # --- Actualización de Velocidad ---
            # Factores aleatorios
            r1_seq = np.random.rand(particle.num_jobs)
            r2_seq = np.random.rand(particle.num_jobs)
            r1_room = np.random.rand(particle.num_jobs * 3)
            r2_room = np.random.rand(particle.num_jobs * 3)

            # --- Velocidad de Secuencia (basada en rangos/orden) ---
            def get_rank_vector(sequence, job_ids_map):
                """Convierte una secuencia de jobs a un vector de rangos."""
                rank_vec = np.zeros(len(job_ids_map))
                job_to_rank = {job_id: rank for rank, job_id in enumerate(sequence)}
                for job_id, index in job_ids_map.items():
                    # Usar num_jobs como rango si el job no está en la secuencia (caso raro)
                    rank_vec[index] = job_to_rank.get(job_id, len(job_ids_map))
                return rank_vec

            current_rank = get_rank_vector(particle.position_sequence, job_id_to_index)
            pbest_rank = get_rank_vector(particle.pbest_sequence, job_id_to_index)
            gbest_rank = get_rank_vector(gbest_sequence, job_id_to_index)

            particle.velocity_seq = (
                W_DPSO * particle.velocity_seq +
                C1_DPSO * r1_seq * (pbest_rank - current_rank) +
                C2_DPSO * r2_seq * (gbest_rank - current_rank)
            )

            # --- Velocidad de Asignación de Salas (vector plano normalizado) ---
            def get_room_vector(room_dict, job_ids_list, aprs, ors, arrs):
                """Convierte asignación de salas a vector numérico normalizado."""
                vec = np.zeros(len(job_ids_list) * 3)
                room_lists = [aprs, ors, arrs]
                room_indices = {
                    room: idx / (len(lst) - 1) if len(lst) > 1 else 0.5
                    for op_idx, lst in enumerate(room_lists)
                    for idx, room in enumerate(lst)
                }

                current_idx = 0
                for job_id in job_ids_list:
                    job_rooms = room_dict.get(job_id, {})
                    for op in range(1, 4):
                        room_name = job_rooms.get(op)
                        # Usar 0.5 (medio) si falta la sala o no está en la lista esperada
                        vec[current_idx] = room_indices.get(room_name, 0.5)
                        current_idx += 1
                return vec

            current_rooms_vec = get_room_vector(
                particle.position_rooms, job_ids, aprs, ors, arrs
            )
            pbest_rooms_vec = get_room_vector(
                particle.pbest_rooms, job_ids, aprs, ors, arrs
            )
            gbest_rooms_vec = get_room_vector(
                gbest_rooms, job_ids, aprs, ors, arrs
            )

            particle.velocity_rooms_flat = (
                W_DPSO * particle.velocity_rooms_flat +
                C1_DPSO * r1_room * (pbest_rooms_vec - current_rooms_vec) +
                C2_DPSO * r2_room * (gbest_rooms_vec - current_rooms_vec)
            )

            # Limitar velocidad
            particle.velocity_seq = np.clip(
                particle.velocity_seq, VEL_LOW_DPSO, VEL_HIGH_DPSO
            )
            particle.velocity_rooms_flat = np.clip(
                particle.velocity_rooms_flat, VEL_LOW_DPSO, VEL_HIGH_DPSO
            )

            # --- Actualización de Posición ---

            # 1. Actualizar Secuencia (usando swap basado en sigmoide de velocidad)
            # Convertir velocidad a probabilidad de swap
            prob_swap = sigmoid(particle.velocity_seq)
            # Iterar sobre los índices de la secuencia
            indices = list(range(particle.num_jobs))
            random.shuffle(indices) # Procesar en orden aleatorio
            for k in indices:
                if random.random() < prob_swap[k]:
                    # Elegir otro índice 'l' diferente de 'k' para intercambiar
                    if particle.num_jobs >= 2:
                        l = random.choice([idx for idx in range(particle.num_jobs) if idx != k])
                        # Realizar el swap
                        (particle.position_sequence[k], particle.position_sequence[l]) = \
                        (particle.position_sequence[l], particle.position_sequence[k])


            # 2. Actualizar Asignación de Salas (copiando de pbest/gbest basado en sigmoide)
            prob_room_change = sigmoid(particle.velocity_rooms_flat)
            room_idx = 0
            for job_id in job_ids:
                if job_id not in particle.position_rooms: continue
                for op in range(1, 4):
                    if op not in particle.position_rooms[job_id]: continue

                    # Decidir si cambiar esta asignación de sala
                    if random.random() < prob_room_change[room_idx]:
                        # Decidir si copiar de pbest o gbest
                        copy_from_pbest = (job_id in particle.pbest_rooms and
                                        op in particle.pbest_rooms[job_id])
                        copy_from_gbest = (gbest_rooms is not None and
                                        job_id in gbest_rooms and
                                        op in gbest_rooms[job_id])

                        # Probabilidad de usar pbest vs gbest
                        use_pbest_prob = C1_DPSO / (C1_DPSO + C2_DPSO) if (C1_DPSO + C2_DPSO) > 0 else 0.5
                        source_rooms = None

                        if random.random() < use_pbest_prob:
                            if copy_from_pbest:
                                source_rooms = particle.pbest_rooms
                            elif copy_from_gbest: # Fallback a gbest si pbest no es aplicable
                                source_rooms = gbest_rooms
                        else: # Intentar gbest primero
                            if copy_from_gbest:
                                source_rooms = gbest_rooms
                            elif copy_from_pbest: # Fallback a pbest si gbest no es aplicable
                                source_rooms = particle.pbest_rooms

                        # Si se encontró una fuente válida, copiar la asignación
                        if source_rooms:
                            particle.position_rooms[job_id][op] = source_rooms[job_id][op]

                    room_idx += 1
                    if room_idx >= len(prob_room_change): break # Evitar error de índice
                if room_idx >= len(prob_room_change): break # Evitar error de índice

    print(" ¡Hecho!") # Fin de las iteraciones

    # Crear una representación de la mejor partícula encontrada
    best_particle_final = None
    if gbest_sequence is not None and gbest_rooms is not None:
        best_particle_final = DiscreteParticle(
            job_ids, aprs, ors, arrs, VEL_LOW_DPSO, VEL_HIGH_DPSO
        )
        best_particle_final.position_sequence = gbest_sequence[:]
        best_particle_final.position_rooms = copy.deepcopy(gbest_rooms)
        best_particle_final.current_value = gbest_value
        best_particle_final.pbest_value = gbest_value # pbest = gbest para la partícula final
        best_particle_final.pbest_sequence = gbest_sequence[:]
        best_particle_final.pbest_rooms = copy.deepcopy(gbest_rooms)

    return gbest_value, best_particle_final, best_fitness_history, avg_fitness_history

# %%
# --- 9. FUNCIONES DE IMPRESIÓN Y EXPORTACIÓN DE HORARIOS ---
import csv

def print_schedule_summary(schedule_details, title):
    if not schedule_details: return
    print(f"\n--- Schedule Summary: {title} ---")
    header = f"{'Job (E)':<10s} | {'Operation':^10s} | {'Resource':<12s} | {'Start':>10s} | {'Finish':>10s} | {'Duration':>10s}"
    print(header); print("-" * len(header))
    for task in sorted(schedule_details, key=lambda x: x['Start']):
        job_label = f"{task['Job']} (E)" if task['Job'] in emergency_jobs else str(task['Job'])
        print(f"{job_label:<10s} | {task['Operation']:^10d} | {task['Resource']:<12s} | {task['Start']:>10.2f} | {task['Finish']:>10.2f} | {task['Finish'] - task['Start']:>10.2f}")
    print("-" * len(header))

def display_scheduling_strategy(schedule_details):
    if not schedule_details: return
    print("\n\n--- Scheduling Strategy by Operating Room ---")
    room_schedules = {room: [] for room in all_rooms}
    for task in schedule_details:
        room_schedules[task['Resource']].append((task['Start'], task['Job'], task['Operation']))

    print(f"{'O.R.':<12} | {'Sequence of Operations (Job(Op))'}")
    print("-" * 80)
    for room_name in sorted(room_schedules.keys()):
        schedule = sorted(room_schedules[room_name], key=lambda x: x[0])
        sequence_str = [f"{j}{'(E)' if j in emergency_jobs else ''}(Op{o})" for _, j, o in schedule]
        print(f"{room_name:<12} | {' -> '.join(sequence_str)}")
    print("-" * 80)

def display_starting_timetable(schedule_details):
    if not schedule_details: return
    print("\n\n--- Start Times by Surgery ---")
    job_timetables = {}
    for task in schedule_details:
        job_id = task['Job']
        if job_id not in job_timetables: job_timetables[job_id] = {}
        job_timetables[job_id][task['Operation']] = task['Start']

    header = f"{'Surgery (E)':<15} | {'Anesthesia Start (min)':>25} | {'Surgery Start (min)':>25} | {'Recovery Start (min)':>25}"
    print(header); print("-" * len(header))
    for job_id in sorted(job_timetables.keys()):
        timings = job_timetables.get(job_id, {})
        job_label = f"{job_id} {'(Emergency)' if job_id in emergency_jobs else ''}"
        print(f"{job_label:<15} | {timings.get(1, -1):>25.2f} | {timings.get(2, -1):>25.2f} | {timings.get(3, -1):>25.2f}")
    print("-" * len(header))

def export_schedule_to_csv(schedule_details, filename):
    if not schedule_details:
        print(f"Warning: No schedule details to export to {filename}.")
        return
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = schedule_details[0].keys()
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for row in schedule_details:
                writer.writerow(row)
        print(f"Schedule successfully exported to {filename}")
    except (IOError, IndexError) as e:
        print(f"Error exporting to CSV: {e}")

def export_strategy_to_csv(schedule_details, filename):
    if not schedule_details:
        print(f"Warning: No schedule details to export strategy to {filename}.")
        return
    room_schedules = {room: [] for room in all_rooms}
    for task in schedule_details:
        room_schedules[task['Resource']].append((task['Start'], task['Job'], task['Operation']))

    try:
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Operating_Room', 'Operation_Sequence'])
            for room_name in sorted(room_schedules.keys()):
                schedule = sorted(room_schedules[room_name], key=lambda x: x[0])
                sequence_str = ' -> '.join([f"{j}{'(E)' if j in emergency_jobs else ''}(Op{o})" for _, j, o in schedule])
                writer.writerow([room_name, sequence_str])
        print(f"Strategy successfully exported to {filename}")
    except IOError as e:
        print(f"Error exporting strategy to CSV: {e}")

def export_timetable_to_csv(schedule_details, filename):
    if not schedule_details:
        print(f"Warning: No schedule details to export timetable to {filename}.")
        return
    job_timetables = {}
    for task in schedule_details:
        job_id = task['Job']
        if job_id not in job_timetables:
            job_timetables[job_id] = {}
        job_timetables[job_id][task['Operation']] = task['Start']

    try:
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Surgery_ID', 'Is_Emergency', 'Anesthesia_Start_Time', 'Surgery_Start_Time', 'Recovery_Start_Time'])
            for job_id in sorted(job_timetables.keys()):
                timings = job_timetables.get(job_id, {})
                is_emergency = job_id in emergency_jobs
                row = [
                    job_id,
                    is_emergency,
                    timings.get(1, -1),
                    timings.get(2, -1),
                    timings.get(3, -1)
                ]
                writer.writerow(row)
        print(f"Timetable successfully exported to {filename}")
    except IOError as e:
        print(f"Error exporting timetable to CSV: {e}")

def export_paired_test_report_to_txt(
    filename,
    *,
    mean_ga, sd_ga, n_ga,
    mean_dpso, sd_dpso, n_dpso,
    delta, ci_low, ci_high,
    w_stat, p_value, cohens_dz,
    alpha_test=0.05,
    algos=("GA","dPSO"),
    conclusion_text=None,   # si lo pasas, se usa tal cual
    pvalue_csv_path=None,   # si lo pasas, se menciona al final
):
    """
    Escribe un reporte completo del análisis estadístico pareado en un .txt
    (resumen descriptivo, contraste, conclusión y tabla simbólica de p-values).
    """
    # preparar directorio
    outdir = os.path.dirname(filename)
    if outdir:
        os.makedirs(outdir, exist_ok=True)

    # conclusión
    if conclusion_text is None:
        if p_value < alpha_test:
            if delta < 0:
                conclusion = "→ Conclusión: GA obtiene makespan significativamente menor que dPSO (mejor desempeño)."
            elif delta > 0:
                conclusion = "→ Conclusión: dPSO obtiene makespan significativamente menor que GA (mejor desempeño)."
            else:
                conclusion = "→ Conclusión: Diferencia estadísticamente significativa, pero Δ≈0 (revisar datos)."
        else:
            conclusion = f"→ Conclusión: No hay evidencia suficiente de diferencia estadística (α = {alpha_test}).\n"
    else:
        conclusion = conclusion_text

    # símbolo de la tabla
    p_symbol = ">= 0.05" if p_value >= alpha_test else "< 0.05"

    # armar texto
    lines = []
    lines.append("------------------------- ANÁLISIS ESTADÍSTICO (PAREADO) -------------------------")
    lines.append("Resumen descriptivo:")
    lines.append(f"  - GA:   media = {mean_ga:.4f} min | sd = {sd_ga:.4f} | n = {int(n_ga)}")
    lines.append(f"  - dPSO: media = {mean_dpso:.4f} min | sd = {sd_dpso:.4f} | n = {int(n_dpso)}")
    lines.append("")
    lines.append("Contraste pareado:")
    lines.append(f"  - Δ = GA - dPSO = {delta:.4f} min (IC95% [{ci_low:.4f}, {ci_high:.4f}])")
    lines.append(f"  - Wilcoxon pareado: W = {w_stat:.4f}, p = {p_value:.4f}")
    lines.append(f"  - Tamaño de efecto (Cohen's dz) = {cohens_dz:.3f}")
    lines.extend(conclusion.split("\n"))
    lines.append("")
    lines.append("Tabla de P-value promedio (Fila vs. Columna)")

    # tabla con columnas alineadas
    colw = 10
    header = " " * 8 + "".join([f"{a:<{colw}}" for a in algos])
    lines.append(header)
    for r in algos:
        row_cells = []
        for c in algos:
            cell = "-" if r == c else p_symbol
            row_cells.append(f"{cell:<{colw}}")
        lines.append(f"{r:<8}" + "".join(row_cells))

    if pvalue_csv_path:
        lines.append(f"Tabla de P-value exportada exitosamente a {pvalue_csv_path}")

    # escribir archivo
    with open(filename, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

def export_summary_statistics_to_csv(results_list, num_simulations, filename):
    """
    Exporta las estadísticas comparativas (mínimo, promedio, std) a un archivo CSV.
    'results_list' debe ser una lista de tuplas, ej: [('GA', ga_valid_makespans), ('dPSO', dpso_valid_makespans)]
    """
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)

            # Escribir la cabecera del CSV
            writer.writerow([
                'algorithm', 'valid_simulations', 'total_simulations',
                'minimum_makespan', 'average_makespan', 'std_makespan'
            ])

            # Procesar y escribir los datos de cada algoritmo
            for algo_name, makespan_data in results_list:
                if makespan_data: # Solo escribir si hay datos válidos
                    valid_count = len(makespan_data)
                    min_makespan = np.min(makespan_data)
                    mean_makespan = np.mean(makespan_data)
                    std_makespan = np.std(makespan_data)

                    writer.writerow([
                        algo_name, valid_count, num_simulations,
                        f"{min_makespan:.2f}", f"{mean_makespan:.2f}", f"{std_makespan:.2f}"
                    ])

        print(f"Estadísticas comparativas exportadas exitosamente a {filename}")

    except IOError as e:
        print(f"Error exportando las estadísticas a CSV: {e}")

# %%
# --- 10. Bucle Monte Carlo para Comparación ---

os.makedirs("Actualizado/graphs/boxplot", exist_ok=True)
os.makedirs("Actualizado/graphs/convergence", exist_ok=True)
os.makedirs("Actualizado/graphs/gantt", exist_ok=True)
os.makedirs("Actualizado/graphs/csv", exist_ok=True)
os.makedirs("Actualizado/graphs/histograms", exist_ok=True)

NUM_SIMULATIONS = 300  # Número de simulaciones Monte Carlo

# Almacenamiento de resultados
ga_results = {'obj': [], 'makespan': [], 'solution': [], 'best_hist': [], 'avg_hist': []}
dpso_results = {'obj': [], 'makespan': [], 'solution': [], 'best_hist': [], 'avg_hist': []}

# Almacenamiento para detalles del mejor run global
best_ga_makespan_overall = float('inf')
best_ga_solution_repr = None
best_ga_data_seed = -1

best_dpso_makespan_overall = float('inf')
best_dpso_solution_repr = None # Guardará la partícula DiscreteParticle
best_dpso_data_seed = -1

std_factor = 0.1

ga_times = []
dpso_times = []

print(f"Iniciando {NUM_SIMULATIONS} simulaciones Monte Carlo para GA ...")
start_monte = time.time()

for sim in trange(NUM_SIMULATIONS, desc="Monte Carlo"):
    # Prints solo al inicio, cada 20 simulaciones, y última
    if sim == 0 or (sim + 1) % 20 == 0 or sim == NUM_SIMULATIONS - 1:
        print(f"\n--- Simulación {sim + 1}/{NUM_SIMULATIONS} ---")

    # Semilla para generar datos
    data_seed = sim
    np.random.seed(data_seed)
    current_surgeries_data = generate_day_surgeries_data(
        job_ids, base_day_surgeries_data, std_factor=std_factor
    )

    # Semilla para los algoritmos
    algo_seed = sim * 10

    # --- Run GA ---
    start_ga = time.time()
    (ga_best_obj, ga_best_sol, ga_best_hist, ga_avg_hist) = run_genetic_algorithm(
        current_surgeries_data, personnel_map, job_ids, num_jobs, algo_seed
    )
    ga_time = time.time() - start_ga
    ga_times.append(ga_time)  # acumular tiempos

    if ga_best_sol:
        _, final_mk_ga, _ = calculate_makespan_ga(
            ga_best_sol, current_surgeries_data, personnel_map, return_details=True
        )
        if final_mk_ga != float('inf'):
            ga_results['obj'].append(ga_best_obj)
            ga_results['makespan'].append(final_mk_ga)
            if final_mk_ga < best_ga_makespan_overall:
                best_ga_makespan_overall = final_mk_ga
                best_ga_solution_repr = copy.deepcopy(ga_best_sol)
                best_ga_data_seed = data_seed
        else:
            ga_results['obj'].append(float('inf'))
            ga_results['makespan'].append(float('inf'))
    else:
        ga_results['obj'].append(float('inf'))
        ga_results['makespan'].append(float('inf'))

    ga_results['solution'].append(ga_best_sol)
    ga_results['best_hist'].append(ga_best_hist)
    ga_results['avg_hist'].append(ga_avg_hist)

    # --- Run dPSO ---
    start_dpso = time.time()
    (dpso_best_obj, dpso_best_particle, dpso_best_hist, dpso_avg_hist) = run_dpso_algorithm(
        current_surgeries_data, personnel_map, job_ids, num_jobs, algo_seed
    )
    dpso_time = time.time() - start_dpso
    dpso_times.append(dpso_time)  # acumular tiempos

    if dpso_best_particle:
        _, final_mk_dpso, _ = calculate_makespan_dpso(
            dpso_best_particle, current_surgeries_data, personnel_map, return_details=True
        )
        if final_mk_dpso != float('inf'):
            dpso_results['makespan'].append(final_mk_dpso)
            if final_mk_dpso < best_dpso_makespan_overall:
                best_dpso_makespan_overall = final_mk_dpso
                best_dpso_solution_repr = copy.deepcopy(dpso_best_particle)
                best_dpso_data_seed = data_seed
        else:
            dpso_results['makespan'].append(float('inf'))
    else:
        dpso_results['makespan'].append(float('inf'))

    dpso_results['solution'].append(dpso_best_particle)
    dpso_results['best_hist'].append(dpso_best_hist)
    dpso_results['avg_hist'].append(dpso_avg_hist)

end_monte = time.time()

print(f"\n--- Monte Carlo Finalizado ({NUM_SIMULATIONS} simulaciones) ---")
print(f"Tiempo total Monte Carlo: {end_monte - start_monte:.2f} segundos")

# --- Estadísticas de tiempos ---
ga_times = np.array(ga_times)
dpso_times = np.array(dpso_times)

print("\nResumen tiempos GA (s): "
      f"mean={ga_times.mean():.3f}, std={ga_times.std(ddof=1):.3f}, "
      f"min={ga_times.min():.3f}, max={ga_times.max():.3f}")

print("Resumen tiempos dPSO (s): "
      f"mean={dpso_times.mean():.3f}, std={dpso_times.std(ddof=1):.3f}, "
      f"min={dpso_times.min():.3f}, max={dpso_times.max():.3f}")
end_monte = time.time()
print(f"\n--- Monte Carlo Finalizado ({NUM_SIMULATIONS} simulaciones) ---")
print(f"Tiempo total Monte Carlo: {end_monte - start_monte:.2f} segundos\n")

# %%
# --- 11. Estadísticas y Gráficos Comparativos ---

# Filtrar makespans válidos (no infinitos)
ga_valid_makespans = [m for m in ga_results['makespan'] if m != float('inf')]
dpso_valid_makespans = [m for m in dpso_results.get('makespan', []) if m != float('inf')]

print("--- Estadísticas Comparativas ---")
print(f"Resultados basados en {NUM_SIMULATIONS} simulaciones.")

# Estadísticas GA y dPSO
print("--- Estadísticas Comparativas ---")
if ga_valid_makespans:
    print(f"\nAlgoritmo Genético (GA): Válidas: {len(ga_valid_makespans)}/{NUM_SIMULATIONS}")
    print(f"  Makespan Mínimo:      {np.min(ga_valid_makespans):.2f} min")
    print(f"  Makespan Promedio:    {np.mean(ga_valid_makespans):.2f} ± {np.std(ga_valid_makespans):.2f} min")
else:
    print("\nGA: No se obtuvieron resultados válidos.")
if dpso_valid_makespans:
    print(f"\nDiscrete PSO (dPSO): Válidas: {len(dpso_valid_makespans)}/{NUM_SIMULATIONS}")
    print(f"  Makespan Mínimo:      {np.min(dpso_valid_makespans):.2f} min")
    print(f"  Makespan Promedio:    {np.mean(dpso_valid_makespans):.2f} ± {np.std(dpso_valid_makespans):.2f} min")
else:
    print("\ndPSO: No se obtuvieron resultados válidos.")

# --- Exportar Estadísticas a CSV ---
export_summary_statistics_to_csv([('GA', ga_valid_makespans), ('dPSO', dpso_valid_makespans)], NUM_SIMULATIONS, "Actualizado/graphs/csv/summary_statistics.csv")

# Prueba T P-Value
print("\n" + "-"*25 + " ANÁLISIS ESTADÍSTICO (PAREADO) " + "-"*25)

if ga_valid_makespans and dpso_valid_makespans and len(ga_valid_makespans) > 1 and len(dpso_valid_makespans) > 1:
    ga = np.asarray(ga_valid_makespans, dtype=float)
    dpso = np.asarray(dpso_valid_makespans, dtype=float)

    # Asegurar apareamiento: truncar a la longitud mínima si difieren
    n = min(len(ga), len(dpso))

    # Diferencias pareadas (negativo favorece a GA, porque menor makespan es mejor)
    diff = ga - dpso
    mean_ga, mean_dpso = ga.mean(), dpso.mean()
    std_ga, std_dpso = ga.std(ddof=1), dpso.std(ddof=1)
    mean_diff = diff.mean()
    sd_diff = diff.std(ddof=1)

    # Wilcoxon pareado, bilateral (ignora diferencias exactamente 0)
    try:
        w_stat, p_value = stats.wilcoxon(ga, dpso, alternative="two-sided", zero_method="wilcox", method="auto")
    except ValueError:
        # Caso extremo: todas las diferencias son 0
        w_stat, p_value = 0.0, 1.0

    # IC95% del promedio de la diferencia con bootstrap
    rng = np.random.default_rng(42)
    B = 5000
    if n > 1:
        idxs = rng.integers(0, n, size=(B, n))
        boot_means = diff[idxs].mean(axis=1)
        ci_low, ci_high = np.percentile(boot_means, [2.5, 97.5])
    else:
        ci_low = ci_high = mean_diff  # con un solo par no hay IC informativo

    # Tamaño de efecto (Cohen's dz para datos pareados)
    cohens_dz = mean_diff / sd_diff if sd_diff > 0 else np.nan

    # --- Reporte ---
    print("Resumen descriptivo:")
    print(f"  - GA:   media = {mean_ga:.4f} min | sd = {std_ga:.4f} | n = {n}")
    print(f"  - dPSO: media = {mean_dpso:.4f} min | sd = {std_dpso:.4f} | n = {n}\n")

    print("Contraste pareado:")
    print(f"  - Δ = GA - dPSO = {mean_diff:.4f} min (IC95% [{ci_low:.4f}, {ci_high:.4f}])")
    print(f"  - Wilcoxon pareado: W = {w_stat:.4f}, p = {p_value:.4f}")
    print(f"  - Tamaño de efecto (Cohen's dz) = {cohens_dz:.3f}")

    # Conclusión interpretando signo y significancia
    if p_value < alpha_test:
        if mean_ga < mean_dpso:
            print("  → Conclusión: GA obtiene makespan significativamente menor que dPSO (mejor desempeño).")
        else:
            print("  → Conclusión: dPSO obtiene makespan significativamente menor que GA (mejor desempeño).")
    else:
        print("  → Conclusión: No hay evidencia suficiente de diferencia estadística (α = 0.05).")

    # --- Tabla de P-value (promedio fila vs columna) ---
    print("\nTabla de P-value promedio (Fila vs. Columna)")
    p_symbol = "< 0.05" if p_value < alpha_test else ">= 0.05"
    print(f"{'':<8}{'GA':<10}{'dPSO':<10}")
    print(f"{'GA':<8}{'-':<10}{p_symbol:<10}")
    print(f"{'dPSO':<8}{p_symbol:<10}{'-':<10}")

    pvalue_csv_path = "Actualizado/graphs/csv/p_value_analysis.csv"

    # Exportar CSV (usa tu helper existente)
    export_paired_test_report_to_txt(
        "Actualizado/graphs/txt/paired_analysis.txt",
        mean_ga=mean_ga, sd_ga=std_ga, n_ga=n,
        mean_dpso=mean_dpso, sd_dpso=std_dpso, n_dpso=n,
        delta=mean_diff, ci_low=ci_low, ci_high=ci_high,
        w_stat=w_stat, p_value=p_value, cohens_dz=cohens_dz,
        alpha_test=alpha_test,
        algos=("GA","dPSO"),
        # Si quieres forzar una conclusión personalizada, pásala aquí como string:
        conclusion_text=None,
        pvalue_csv_path=pvalue_csv_path,
    )

else:
    print("\nNo hay suficientes datos válidos de ambos algoritmos para realizar el análisis pareado.")

print("-" * 75)

# --- Gráficos ---

def plot_evolution(best_history, avg_history, max_iters_or_gens, algo_name, sim_num):
    """Genera gráfico de evolución del fitness (mejor y promedio)."""
    plt.figure(figsize=(10, 6))

    # Filtrar infinitos para graficar
    valid_best = [(i, f) for i, f in enumerate(best_history) if f != float('inf')]
    valid_avg = [(i, f) for i, f in enumerate(avg_history) if f != float('inf')]

    if valid_best:
        iters_best, vals_best = zip(*valid_best)
        plt.plot(iters_best, vals_best, label=f'Best fitness ({algo_name})',
                 color='blue', linestyle='-', drawstyle='steps-post')
    if valid_avg:
        iters_avg, vals_avg = zip(*valid_avg)
        plt.plot(iters_avg, vals_avg, label=f'Average fitness ({algo_name})',
                 color='orangered', linestyle='--')

    plt.xlabel('Iteration / Generation')
    plt.ylabel('Objective value')
    plt.title(f'{algo_name} evolution (Simulation #{sim_num})')
    plt.xlim(0, max_iters_or_gens)

    # Ajustar límites Y basado en valores finitos
    finite_vals = [f for f in best_history + avg_history if f != float('inf')]
    if finite_vals:
        min_val, max_val = min(finite_vals), max(finite_vals)
        y_padding = (max_val - min_val) * 0.1 if max_val > min_val else 1
        plt.ylim(max(0, min_val - y_padding), max_val + y_padding)

    # Configurar ticks X e Y para mejor visualización
    x_tick_step = max(1, max_iters_or_gens // 10)
    plt.xticks(np.arange(0, max_iters_or_gens + 1, x_tick_step))

    plt.grid(True, linestyle=':', alpha=0.7)
    plt.legend()
    plt.tight_layout()

    plt.savefig(f"Actualizado/graphs/convergence/{algo_name.lower()}_convergence_sim_{sim_num}.png")

    plt.close()

def plot_histogram(results_data, algo_name, num_simulations, bar_color):
    """
    Genera y guarda un histograma para la distribución de resultados de un algoritmo.
    """
    plt.figure(figsize=(10, 6))
    plt.hist(results_data, bins='auto', color=bar_color, alpha=0.9, edgecolor='black')
    plt.title(f'Makespan results distribution - {algo_name} (N={num_simulations} simulations)')
    plt.xlabel('Makespan (minutes)')
    plt.ylabel('Frequency (No. of Simulations)')
    plt.grid(axis='y', linestyle=':', alpha=0.7)
    plt.tight_layout()

    # Guarda en la nueva carpeta 'histograms'
    plt.savefig(f"Actualizado/graphs/histograms/histograma_{algo_name.lower()}.png")
    plt.close()
    print(f"\nHistograma para {algo_name} generado y guardado en la carpeta 'histograms'.")



# --- Boxplot Comparativo de Makespan
plt.figure(figsize=(8, 6))
data_to_plot = []
labels = []

# Añadir datos de GA si son válidos
if ga_valid_makespans:
    data_to_plot.append(ga_valid_makespans)
    labels.append('GA')

# Añadir datos de dPSO si son válidos
if dpso_valid_makespans:
    data_to_plot.append(dpso_valid_makespans)
    labels.append('dPSO')

if data_to_plot:
    plt.boxplot(data_to_plot, labels=labels)
    plt.ylabel('Real makespan (minutes)')
    plt.title(f'Makespan comparison (N={NUM_SIMULATIONS})') # Título actualizado
    plt.grid(True, axis='y', linestyle=':', alpha=0.7)
    plt.tight_layout()
    # Nombre de archivo actualizado
    plt.savefig("Actualizado/graphs/boxplot/makespan_comparison_boxplot.png")
    plt.close()
else:
    print("\nNo hay datos válidos para graficar el boxplot.")

# --- Gráfico Evolución Mejor Run GA ---
best_ga_sim_idx = -1
if ga_valid_makespans:
    try:
        best_ga_sim_idx = ga_results['makespan'].index(min(ga_valid_makespans))
    except (ValueError, IndexError):
        pass

if best_ga_sim_idx != -1:
    print(f"\nGenerando gráfico de evolución GA (Mejor Simulación #{best_ga_sim_idx + 1})")
    plot_evolution(
        ga_results['best_hist'][best_ga_sim_idx],
        ga_results['avg_hist'][best_ga_sim_idx],
        MAX_GENERATIONS, "GA", best_ga_sim_idx + 1
    )
else:
    print("\nNo se pudo identificar la mejor simulación GA para graficar evolución.")

# --- Gráfico Evolución Mejor Run dPSO
best_dpso_sim_idx = -1
if dpso_valid_makespans:
    try:
        best_dpso_sim_idx = dpso_results['makespan'].index(min(dpso_valid_makespans))
    except (ValueError, IndexError):
        pass

if best_dpso_sim_idx != -1:
    print(f"\nGenerando gráfico de evolución dPSO (Mejor Simulación #{best_dpso_sim_idx + 1})")
    plot_evolution(
        dpso_results['best_hist'][best_dpso_sim_idx],
        dpso_results['avg_hist'][best_dpso_sim_idx],
        MAX_ITERATIONS_DPSO, "dPSO", best_dpso_sim_idx + 1
    )
else:
    print("\nNo se pudo identificar la mejor simulación dPSO para graficar evolución.")

from matplotlib.patches import Patch

SETUP_COLOR = 'yellow'
CLEANUP_COLOR = 'lightcoral'

def plot_gantt_chart(schedule_details, all_rooms_ordered, title):
    """
    Genera un diagrama de Gantt con segmentos para Setup, Processing y Cleanup,
    agrupando APR, OR y ARR visualmente con bloques achatados y espaciado automático.
    """
    if not schedule_details:
        print(f"No hay detalles de programación válidos para generar el Gantt: {title}")
        return

    # === CONFIGURACIÓN DE ALTURA ===
    bar_height = 0.25            # Altura visual de cada bloque
    room_spacing = bar_height    # Espacio entre recursos del mismo grupo
    group_gap = bar_height * 3  # Espacio entre grupos (APR/OR/ARR)

    # === COLORMAP PARA JOBS ===
    unique_jobs = sorted(set(task['Job'] for task in schedule_details))
    try:
        colors = plt.cm.get_cmap('tab20', len(unique_jobs))
        if len(unique_jobs) > 20:
            colors = plt.cm.get_cmap('turbo', len(unique_jobs))
    except ValueError:
        colors = plt.cm.get_cmap('tab10')
    job_colors = {job_id: colors(i % colors.N) for i, job_id in enumerate(unique_jobs)}

    fig, ax = plt.subplots(figsize=(11, 6))

    # === CATEGORIZACIÓN DE SALAS ===
    room_categories = {'APR': [], 'OR': [], 'ARR': []}
    for room in all_rooms_ordered:
        if room.startswith('APR'): room_categories['APR'].append(room)
        elif room.startswith('OR'): room_categories['OR'].append(room)
        elif room.startswith('ARR'): room_categories['ARR'].append(room)

    # === POSICIONES Y ===
    y_pos = {}
    y_labels_ordered = []
    current_y = 0

    for group in ['APR', 'OR', 'ARR']:
        for room in sorted(room_categories[group]):
            y_pos[room] = current_y
            y_labels_ordered.append(room)
            current_y += room_spacing
        current_y += group_gap

    max_time = 0
    bar_edge_color = 'black'
    bar_linewidth = 0.5
    bar_alpha = 0.9

    for task in schedule_details:
        job = task['Job']
        resource = task['Resource']
        start = task.get('Start', -1)
        processing_end = task.get('ProcessingEnd', -1)
        finish = task.get('Finish', -1)

        job_type = job_types.get(job, 1)
        setup_time_val = setup_times[job_type]
        cleanup_time_val = cleanup_times[job_type]

        if start < 0 or processing_end < 0 or finish < 0 or processing_end < start or finish < processing_end:
            continue
        if resource not in y_pos:
            continue

        setup_duration = setup_time_val
        processing_duration = processing_end - (start + setup_duration)
        cleanup_duration = cleanup_time_val

        if setup_duration < 0 or processing_duration < 0 or cleanup_duration < 0:
            continue

        setup_start = start
        processing_start = start + setup_duration
        cleanup_start = processing_end
        y = y_pos[resource]
        processing_color = job_colors.get(job, 'gray')

        if setup_duration > 1e-6:
            ax.barh(y=y, width=setup_duration, left=setup_start, height=bar_height,
                    color=SETUP_COLOR, edgecolor=bar_edge_color, linewidth=bar_linewidth, alpha=bar_alpha)

        if processing_duration > 1e-6:
            ax.barh(y=y, width=processing_duration, left=processing_start, height=bar_height,
                    color=processing_color, edgecolor=bar_edge_color, linewidth=bar_linewidth, alpha=bar_alpha)

            try:
                bar_color_rgb = mcolors.to_rgb(processing_color)
                luminance = 0.299*bar_color_rgb[0] + 0.587*bar_color_rgb[1] + 0.114*bar_color_rgb[2]
                text_color = 'white' if luminance < 0.5 else 'black'
            except:
                text_color = 'black'

            if processing_duration > 20:
                ax.text(processing_start + processing_duration / 2, y, f"{job}",
                        ha='center', va='center', color=text_color,
                        fontweight='bold', fontsize=7, clip_on=True)

        if cleanup_duration > 1e-6:
            ax.barh(y=y, width=cleanup_duration, left=cleanup_start, height=bar_height,
                    color=CLEANUP_COLOR, edgecolor=bar_edge_color, linewidth=bar_linewidth, alpha=bar_alpha)

        if finish > max_time:
            max_time = finish

    # === EJE Y ===
    ax.set_yticks([y_pos[room] for room in y_labels_ordered])
    ax.set_yticklabels(y_labels_ordered, fontsize=9)
    ax.set_xlabel("Time (minutes)", fontsize=10)
    ax.set_ylabel("Resource (Room)", fontsize=10)
    ax.set_title(title + " (Setup/Proc/Cleanup)", fontsize=12, fontweight='bold')

    # === SEPARADORES ENTRE GRUPOS (opcionales) ===
    apr_count = len(room_categories['APR'])
    or_count = len(room_categories['OR'])
    sep_lines = [y_pos[room_categories['OR'][0]] - group_gap/2,
                 y_pos[room_categories['ARR'][0]] - group_gap/2]
    for sep in sep_lines:
        ax.axhline(y=sep, color='black', linestyle='--', linewidth=0.6, alpha=0.4)

    # === EJE X Y ESTILO ===
    ax.set_xlim(0, max_time * 1.05 if max_time > 0 else 100)
    ax.invert_yaxis()
    ax.grid(True, axis='x', linestyle=':', color='gray', alpha=0.6)

    # === LEYENDA ===
    legend_elements = [
        Patch(facecolor=SETUP_COLOR, edgecolor=bar_edge_color, label='Setup'),
        Patch(facecolor='grey', edgecolor=bar_edge_color, label='Processing (Color per Job)'),
        Patch(facecolor=CLEANUP_COLOR, edgecolor=bar_edge_color, label='Cleanup')
    ]
    ax.legend(handles=legend_elements, loc='upper right', fontsize='small', title="Segments:")

    last_y_value = max(y_pos.values())
    #ax.set_ylim(last_y_value + 0.12, -0.12)  # +1 para evitar cortar parte inferior

    # === AJUSTE FINAL DE LAYOUT ===
    plt.tight_layout(pad=0.2)

    # === GUARDAR ===
    safe_title = title.replace(" ", "_").replace(":", "").replace("(", "").replace(")", "").lower()
    filename = f"Actualizado/graphs/gantt/{safe_title}.png"
    plt.savefig(filename)
    plt.close()
    print(f"Gantt chart saved to {filename}")

# %%
# --- 12. GENERACIÓN DE GRÁFICOS GANTT PARA LOS MEJORES RUNS ---

print("\n--- Generando Gráficos Gantt para las Mejores Simulaciones Globales ---")

# Mejor Run GA Global
if best_ga_solution_repr and best_ga_data_seed != -1:
    print(f"\nGenerando Gantt para el mejor GA global...")
    print(f"  Makespan: {best_ga_makespan_overall:.2f} min")
    print(f"  Obtenido en simulación con semilla de datos: {best_ga_data_seed}")

    # Regenerar datos de esa simulación específica
    np.random.seed(best_ga_data_seed)
    best_ga_surgeries_data = generate_day_surgeries_data(job_ids, base_day_surgeries_data, std_factor=std_factor)  # Variabilidad del 0% (ajustable)
    # Recalcular con detalles usando la mejor solución guardada
    _, _, ga_schedule_details = calculate_makespan_ga(
        best_ga_solution_repr, best_ga_surgeries_data, personnel_map, return_details=True
    )

    if ga_schedule_details:
        plot_title_ga = (f"Gantt diagram - Best GA global fitness "
                        f"(Makespan: {best_ga_makespan_overall:.2f} min)")
        plot_gantt_chart(ga_schedule_details, all_rooms, plot_title_ga)
    else:
        print("  Error: No se pudieron obtener los detalles de la programación para el mejor GA.")
else:
    print("\nNo se encontró una solución válida global para GA para generar Gantt.")

# Mejor Run dPSO Global
if best_dpso_solution_repr and best_dpso_data_seed != -1:
    print(f"\nGenerando Gantt para el mejor dPSO global (Makespan: {best_dpso_makespan_overall:.2f} min)...")
    np.random.seed(best_dpso_data_seed)
    best_dpso_surgeries_data = generate_day_surgeries_data(job_ids, base_day_surgeries_data, std_factor=std_factor)
    _, _, dpso_schedule_details = calculate_makespan_dpso(
        best_dpso_solution_repr, best_dpso_surgeries_data, personnel_map, return_details=True
    )
    if dpso_schedule_details:
        plot_title_dpso = (f"Gantt diagram - Best dPSO global fitness (Makespan {best_dpso_makespan_overall:.2f})")
        plot_gantt_chart(dpso_schedule_details, all_rooms, plot_title_dpso)
    else:
        print("Error: No se pudieron obtener los detalles de la programación para el mejor dPSO.")
else:
    print("\nNo se encontró una solución válida global para dPSO para generar Gantt.")

print("\n--- Fin del Script ---")



# %%
# --- 13. IMPRESIÓN Y EXPORTACIÓN DE RESULTADOS FINALES ---

# --- Reportes para el Mejor GA Global ---
# (Verifica que los detalles del GA existen antes de llamar a las funciones)
if 'ga_schedule_details' in locals() and ga_schedule_details:
    print("\n" + "="*25 + " REPORTE MEJOR RESULTADO: GA " + "="*25)
    print_schedule_summary(ga_schedule_details, "GA Mejor Run")
    display_scheduling_strategy(ga_schedule_details)
    display_starting_timetable(ga_schedule_details)

    # Exportar a CSV
    export_schedule_to_csv(ga_schedule_details, "Actualizado/graphs/csv/ga_schedule_detallado.csv")
    export_strategy_to_csv(ga_schedule_details, "Actualizado/graphs/csv/ga_estrategia_salas.csv")
    export_timetable_to_csv(ga_schedule_details, "Actualizado/graphs/csv/ga_horarios_inicio.csv")

# --- Reportes para el Mejor dPSO Global ---
# (Verifica que los detalles del dPSO existen antes de llamar a las funciones)
if 'dpso_schedule_details' in locals() and dpso_schedule_details:
    print("\n" + "="*25 + " REPORTE MEJOR RESULTADO: dPSO " + "="*25)
    print_schedule_summary(dpso_schedule_details, "dPSO Mejor Run")
    display_scheduling_strategy(dpso_schedule_details)
    display_starting_timetable(dpso_schedule_details)

    # Exportar a CSV con nombres de archivo diferentes
    export_schedule_to_csv(dpso_schedule_details, "Actualizado/graphs/csv/dpso_schedule_detallado.csv")
    export_strategy_to_csv(dpso_schedule_details, "Actualizado/graphs/csv/dpso_estrategia_salas.csv")
    export_timetable_to_csv(dpso_schedule_details, "Actualizado/graphs/csv/dpso_horarios_inicio.csv")

# --- Llamadas a la nueva función de histogramas ---
if ga_valid_makespans:
    plot_histogram(ga_valid_makespans, "GA", NUM_SIMULATIONS, 'skyblue')

if dpso_valid_makespans:
    plot_histogram(dpso_valid_makespans, "dPSO", NUM_SIMULATIONS, 'salmon')

Iniciando 300 simulaciones Monte Carlo para GA ...


Monte Carlo:   0%|          | 0/300 [00:00<?, ?it/s]


--- Simulación 1/300 ---
GA - Gen 0... 200... 400... 600... 800... 1000... ¡Hecho!
dPSO - Iter 0... 200... 400... 600... 800...

Monte Carlo:   0%|          | 1/300 [00:14<1:12:30, 14.55s/it]

 1000... ¡Hecho!
GA - Gen 0... 200... 400... 600... 800... 1000... ¡Hecho!
dPSO - Iter 0... 200... 400... 600... 800...

...


Generando gráfico de evolución dPSO (Mejor Simulación #107)

--- Generando Gráficos Gantt para las Mejores Simulaciones Globales ---

Generando Gantt para el mejor GA global...
  Makespan: 1100.57 min
  Obtenido en simulación con semilla de datos: 106


/tmp/ipython-input-1057971950.py:1660: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab20', len(unique_jobs))


Gantt chart saved to Actualizado/graphs/gantt/gantt_diagram_-_best_ga_global_fitness_makespan_1100.57_min.png

Generando Gantt para el mejor dPSO global (Makespan: 1095.28 min)...


/tmp/ipython-input-1057971950.py:1660: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab20', len(unique_jobs))


Gantt chart saved to Actualizado/graphs/gantt/gantt_diagram_-_best_dpso_global_fitness_makespan_1095.28.png

--- Fin del Script ---

========================= REPORTE MEJOR RESULTADO: GA =========================

--- Schedule Summary: GA Mejor Run ---
Job (E)    | Operation  | Resource     |      Start |     Finish |   Duration
-----------------------------------------------------------------------------
10         |     1      | APR_1        |       0.00 |      82.34 |      82.34
8          |     1      | APR_2        |       0.00 |     113.56 |     113.56
5          |     1      | APR_3        |       0.00 |     130.89 |     130.89
2          |     1      | APR_4        |       0.00 |      92.66 |      92.66
10         |     2      | OR_2         |      62.34 |     193.17 |     130.83
2          |     2      | OR_4         |      72.66 |     175.62 |     102.96
15         |     1      | APR_1        |      82.34 |     213.90 |     131.56
9          |     1      | APR_4        |     

In [5]:
from psutil import *
# This code will return the number of CPU
print("Number of CPU: ", cpu_count())
# This code will return the CPU info
!cat /proc/cpuinfo

Number of CPU:  2
processor	: 0
vendor_id	: AuthenticAMD
cpu family	: 23
model		: 49
model name	: AMD EPYC 7B12
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2249.998
cache size	: 512 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good nopl nonstop_tsc cpuid extd_apicid tsc_known_freq pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm cmp_legacy cr8_legacy abm sse4a misalignsse 3dnowprefetch osvw topoext ssbd ibrs ibpb stibp vmmcall fsgsbase tsc_adjust bmi1 avx2 smep bmi2 rdseed adx smap clflushopt clwb sha_ni xsaveopt xsavec xgetbv1 clzero xsaveerptr arat npt nrip_save umip rdpid
bugs		: sysret_ss_attrs null_seg spectre_v1 spectre_v2 spec_store_bypass retbleed

In [9]:
!cat /etc/os-release

PRETTY_NAME="Ubuntu 22.04.4 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.4 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [10]:
!CPU=$(lscpu | grep "Model name" | sed 's/Model name:\s*//')
!CORES=$(lscpu | grep "Core(s) per socket" | awk '{print $4}')
!THREADS=$(lscpu | grep "^CPU(s):" | awk '{print $2}')
!BASE=$(lscpu | grep "CPU MHz" | awk '{print $3}')
!CACHE=$(lscpu | grep "L3 cache" | awk '{print $3 $4}')

!echo "CPU $CPU ($CORES cores, $THREADS threads; base ${BASE}MHz; L3 cache $CACHE)"

CPU  ( cores,  threads; base MHz; L3 cache )


In [14]:
import cpuinfo, psutil
info = cpuinfo.get_cpu_info()
cpu_name = info['brand_raw']


cores = psutil.cpu_count(logical=False)   # físicos
threads = psutil.cpu_count(logical=True)  # lógicos


freq = psutil.cpu_freq()
base = freq.min / 1000 if freq else None  # GHz
maxi = freq.max / 1000 if freq else None  # GHz


cache = info.get('l3_cache_size', 'N/A')

print(f"CPU {cpu_name} ({cores} cores, {threads} threads; "
      f"base {base:.1f} GHz, turbo up to {maxi:.1f} GHz; "
      f"L3 cache {cache})")

CPU AMD EPYC 7B12 (1 cores, 2 threads; base 0.0 GHz, turbo up to 0.0 GHz; L3 cache 524288)
